In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 100)

In [ ]:
columns=['open', 'high', 'low', 'close','volume', 'close_time', 'quote_volume', 'count', 'taker_buy_volume', 'taker_buy_quote_volume', 'ignore']
btc=pd.read_csv('data/BTCUSDT-1s-2024-04.csv',names=columns)
eth=pd.read_csv('data/ETHUSDT-1s-2024-04.csv',names=columns)
usdc=pd.read_csv('data/USDCUSDT-1s-2024-04.csv',names=columns)
# print(btc)
# print(eth)
# print(usdc)

In [ ]:
btc['price']=(btc['open']+btc['close']+btc['high']+btc['low'])/4
eth['price']=(eth['open']+eth['close']+eth['high']+eth['low'])/4
usdc['price']=(usdc['open']+usdc['close']+usdc['high']+usdc['low'])/4
prices=pd.DataFrame({'btceth':btc['price']/eth['price'],'usdteth':1/eth['price'],'usdceth': usdc['price']/eth['price']})
# print(prices)

In [ ]:
wethAddr = '0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2'
wbtcAddr = '0x2260FAC5E5542a773Aa44fBCfeDf7C193bc2C599'
usdcAddr = '0xA0b86991c6218b36c1d19D4a2e9Eb0cE3606eB48'
usdtAddr = '0xdAC17F958D2ee523a2206206994597C13D831ec7'
usdcEth500Addr='0x88e6A0c2dDD26FEEb64F039a2c41296FcB3f5640'
ethUSDT500Addr='0x11b815efB8f581194ae79006d24E0d814B7697F6'
wbtcEth500Addr='0x4585FE77225b41b697C938B018E2Ac67Ac5a20c0'

pools = [
    # [poolAddr,baseAddr,quoteAddr,baseDecimal,quoteDecimal,cexPoolName]
    [usdcEth500Addr,usdcAddr,wethAddr,6,18,'usdceth'],
    [ethUSDT500Addr,usdtAddr,wethAddr,6,18,'usdteth'],
    [wbtcEth500Addr,wbtcAddr,wethAddr,8,18,'btceth']
]

LABELS = {
    '0xa69babef1ca67a37ffaf7a485dfff3382056e78c': 'CEX_SCP',
    '0x51c72848c68a965f66fa7a88855f9f7784502a7f': 'CEX_wintermute',
    '0x1bf621aa9cee3f6154881c25041bb39aed4ca7cc': 'CEX_a7cc',
    '0xe8c060f8052e07423f71d445277c61ac5138a2e5': 'CEX_a2e5',
    '0xbeb5fd030ffb0fbc95d68113c1c796eff65526d7': 'CEX_26d7',
    '0x6f1cdbbb4d53d226cf4b917bf768b94acbab6168': 'CEX_6168',
    '0x9def7cde171841a9f0724124ca0b01a622d749e4': 'CEX_49e4',
    '0x3fc91a3afd70395cd496c647d5a6cc9d4b2b7fad': 'UniversalRouter',
    '0x6b75d8af000000e20b7a7ddf000ba900b4009a80': 'MEV_jaredfromsubway',
    '0x5f515f6c524b18ca30f7783fb58dd4be2e9904ec': '1inch',
    '0xe37e799d5077682fa0a244d46e5649f71457bd09': '1inch',
    '0xe592427a0aece92de3edee1f18e0157c05861564': 'SwapRouter',
    '0x1111111254eeb25477b68fb85ed929f73a960582': 'AggregationRouterV5',
    '0xdef1c0ded9bec7f1a1670819833240f027b25eff': 'ZeroEx',
}


In [ ]:
BUY=0
SELL=1
swaps = pd.read_csv('/data/hackthon/swaps/uni_v3/2024-04-01_bribe.csv')

profits=[]

for bot,label in LABELS.items():
    bot = bot.lower()
    if label[:4]!='CEX_':
        continue
    botswaps = swaps[swaps['to_address']==bot]
    
    for pool in pools:
        bps = botswaps[botswaps['pool_address']==pool[0]]
        if bps.shape[0]==0:
            print(f'{bot} did not trade in {pool[0]}')
            continue
        for _,row in bps.iterrows():
            if row['from_token']==pool[1] and row['to_token']==pool[2]:
                dexSide=SELL
                baseAmount=int(row['from_amount'])/10**pool[3]
                quoteAmount=int(row['to_amount'])/10**pool[4]
            elif row['from_token']==pool[2] and row['to_token']==pool[1]:
                dexSide=BUY
                baseAmount=int(row['to_amount'])/10**pool[3]
                quoteAmount=int(row['from_amount'])/10**pool[4]
            else:
                print(f'unknown swap {row}')
    
            for dt in range(-12,12):
                t = (row['block_ts']+dt)*1000
                if t not in prices.index:
                    continue
                cexPrice = prices.loc[t,pool[5]]
                cexQuoteAmount=baseAmount*cexPrice
                if dexSide==BUY:
                    profit = cexQuoteAmount-quoteAmount
                else:
                    profit = quoteAmount-cexQuoteAmount
                profitRate = profit/quoteAmount

                profits.append({
                    'bot':bot,
                    'pool':pool[0],
                    'dt':dt,
                    'side':dexSide,
                    # 'baseAmount':baseAmount,
                    # 'quoteAmount':quoteAmount,
                    # 'cexPrice':cexPrice,
                    # 'dexPrice':quoteAmount/baseAmount,
                    # 'cexQuoteAmount':cexQuoteAmount,
                    'profit':profit,
                    'profitRate':profitRate,
                    'bribe':row['bribe'],
                    'bribeRate':row['bribe']/profit,
                    'alias':pool[5],
                })

profits=pd.DataFrame(profits)
# print(profits)
                

In [ ]:


for bot,label in LABELS.items():
    bot = bot.lower()
    if label[:4]!='CEX_':
        continue
    botProfit = profits[profits['bot']==bot]
    # for pool in pools:
        # bpp = botProfit[botProfit['pool']==pool[0]]
        # if bpp.shape[0]==0:
            # print(f'{bot} did not trade in {pool[0]}')
            # continue
    # fig,ax = plt.subplots()
    # fig.set_size_inches(20,12)
    fig = go.Figure()
    for t in range(-12,12):
        bptp = botProfit[botProfit['dt']==t]
        # fig = plt.hist(bptp['profitRate'])
        # plt.savefig(f'figs/{bot[-4:]}_{pool[5]}_{t}.png')
        # plt.clf()
        fakeFig,fakeAx = plt.subplots()
        num,bins_limit,patches=fakeAx.hist(bptp['profitRate'],bins=20,range=(-0.001,0.001))
        bins_limit = [(bins_limit[i]+bins_limit[i+1])/2 for i in range(len(bins_limit)-1)]
        fig.add_trace(go.Scatter(x=bins_limit,y=num,name=f'{t}'))
        plt.close(fakeFig)
        # ax.plot(bins_limit,num)
    fig.write_html(f'figs/{bot[-4:]}.html')
    # ax.legend([f'{t}' for t in range(-12,12)])
    # fig.savefig(f'figs/{bot[-4:]}.png',dpi=300)
    # plt.close(fig)

            

In [ ]:
botDelay = {'2a7f':0,'26d7':2,'49e4':1,'6168':1,'a2e5':1000,'a7cc':0,'e78c':-1}


for bot,label in LABELS.items():
    bot = bot.lower()
    if label[:4]!='CEX_':
        continue
    
    dt = botDelay[bot[-4:]]
    botProfit = profits[(profits['bot']==bot) & (profits['dt']==dt) & (profits['profit']>0)]
    # fig = go.Figure()
    # fig.add_trace(go.Histogram(x=botProfit['bribeRate']))
    # fig.update_xaxes(range=[0, 2])
    # fig.write_image(f'figs/{bot[-4:]}_bribe.png',engine='auto')
    
    fig,ax = plt.subplots()
    fig.set_size_inches(20,12)
    ax.hist(botProfit['bribeRate'],bins=20,range=(0,2))
    fig.savefig(f'figs/{bot[-4:]}_bribe.png',dpi=300)
    plt.close(fig)
    
    